<a href="https://colab.research.google.com/github/palVikram/Advance_tensorflow/blob/main/Fine_tuning_xlm_roberta_using_LORA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!unzip /content/jigsaw-toxic-comment-classification-challenge.zip

Archive:  /content/jigsaw-toxic-comment-classification-challenge.zip
  inflating: sample_submission.csv.zip  
  inflating: test.csv.zip            
  inflating: test_labels.csv.zip     
  inflating: train.csv.zip           


In [8]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("/content/train.csv")
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [3]:
df['lenght']=df["comment_text"].apply(lambda x: len(x.split()))

df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,lenght
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,43
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,17
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,42
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,113
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,13
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,47
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,18
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,12
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,25


In [4]:
import re
import string
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove extra whitespace
    text = re.sub(' +', ' ', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Join tokens back into text
    cleaned_text = ' '.join(tokens)

    return cleaned_text

# Example usage:
text = "Hello, this is an example text for cleaning! It contains special characters, 123 and stopwords like 'is' and 'for'."
cleaned_text = clean_text(text)
print("Original text:", text)
print("Cleaned text:", cleaned_text)


Original text: Hello, this is an example text for cleaning! It contains special characters, 123 and stopwords like 'is' and 'for'.
Cleaned text: hello this is an example text for cleaning it contains special characters and stopwords like is and for


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df["comment_text"]=df["comment_text"].apply(lambda x: clean_text(x))

In [6]:
df=df[df["lenght"].apply(lambda x: True if x<=100 else False)]

df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,lenght
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0,43
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0,0,0,0,0,0,17
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0,0,0,0,0,0,42
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0,13
5,00025465d4725e87,congratulations from me as well use the tools ...,0,0,0,0,0,0,13
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,and for the second time of asking when your vi...,0,0,0,0,0,0,47
159567,ffea4adeee384e90,you should be ashamed of yourself that is a ho...,0,0,0,0,0,0,18
159568,ffee36eab5c267c9,spitzer umm theres no actual article for prost...,0,0,0,0,0,0,12
159569,fff125370e4aaaf3,and it looks like it was actually you who put ...,0,0,0,0,0,0,25


In [7]:
df['toxic'].value_counts()

0    117637
1     13735
Name: toxic, dtype: int64

In [8]:
df=df[["comment_text","toxic"]]
df

,comment_text,toxic
0,explanation why the edits made under my userna...,0
1,daww he matches this background colour im seem...,0
2,hey man im really not trying to edit war its j...,0
4,you sir are my hero any chance you remember wh...,0
5,congratulations from me as well use the tools ...,0
...,...,...
159566,and for the second time of asking when your vi...,0
159567,you should be ashamed of yourself that is a ho...,0
159568,spitzer umm theres no actual article for prost...,0
159569,and it looks like it was actually you who put ...,0


In [9]:
#!unzip "/content/archive (3).zip"

In [10]:
df_ol=pd.read_csv("/content/labeled_data.csv")
df_ol

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [11]:
df_ol["tweet"]=df_ol["tweet"].apply(lambda x: clean_text(x))
df_ol

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt mayasolovely as a woman you shouldnt compla...
1,1,3,0,3,0,1,rt mleew boy dats coldtyga dwn bad for cuffin ...
2,2,3,0,3,0,1,rt urkindofbrand dawg rt sbabylife you ever fu...
3,3,3,0,2,1,1,rt cganderson vivabased she look like a tranny
4,4,6,0,6,0,1,rt shenikaroberts the shit you hear about me m...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,yous a muthafin lie lifeasking pearls coreyema...
24779,25292,3,0,1,2,2,youve gone and broke the wrong heart baby and ...
24780,25294,3,0,3,0,1,young buck wan na eat dat nigguh like i aint f...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [12]:
df_ol=df_ol[(df_ol['class']==2) | (df_ol['class']==1)]
df_ol=df_ol[['class', 'tweet']]

df_ol

,class,tweet
0,2,rt mayasolovely as a woman you shouldnt compla...
1,1,rt mleew boy dats coldtyga dwn bad for cuffin ...
2,1,rt urkindofbrand dawg rt sbabylife you ever fu...
3,1,rt cganderson vivabased she look like a tranny
4,1,rt shenikaroberts the shit you hear about me m...
...,...,...
24778,1,yous a muthafin lie lifeasking pearls coreyema...
24779,2,youve gone and broke the wrong heart baby and ...
24780,1,young buck wan na eat dat nigguh like i aint f...
24781,1,youu got wild bitches tellin you lies


In [13]:
df_ol[df_ol["class"]==2]=0
df_ol

,class,tweet
0,0,0
1,1,rt mleew boy dats coldtyga dwn bad for cuffin ...
2,1,rt urkindofbrand dawg rt sbabylife you ever fu...
3,1,rt cganderson vivabased she look like a tranny
4,1,rt shenikaroberts the shit you hear about me m...
...,...,...
24778,1,yous a muthafin lie lifeasking pearls coreyema...
24779,0,0
24780,1,young buck wan na eat dat nigguh like i aint f...
24781,1,youu got wild bitches tellin you lies


In [14]:
df=df[["toxic", "comment_text"]]

df_ol = df_ol.rename(columns={'class': 'toxic', 'tweet': 'comment_text'})


In [15]:
df

,toxic,comment_text
0,0,explanation why the edits made under my userna...
1,0,daww he matches this background colour im seem...
2,0,hey man im really not trying to edit war its j...
4,0,you sir are my hero any chance you remember wh...
5,0,congratulations from me as well use the tools ...
...,...,...
159566,0,and for the second time of asking when your vi...
159567,0,you should be ashamed of yourself that is a ho...
159568,0,spitzer umm theres no actual article for prost...
159569,0,and it looks like it was actually you who put ...


In [16]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df['toxic'] == 0]
df_minority = df[df['toxic'] == 1]

# Upsample minority class
df_majority = resample(df_majority,
                                 replace=True,     # Sample with replacement
                                 n_samples=40000,  # Match number of majority class samples
                                 random_state=42)  # Reproducible results


# Combine majority class with upsampled minority class
df_balanced = pd.concat([df_majority, df_minority])

In [17]:
df_balanced['toxic'].value_counts()

0    40000
1    13735
Name: toxic, dtype: int64

In [18]:
df_balanced = pd.concat([df_balanced, df_ol])

df_balanced

,toxic,comment_text
21447,0,note to wiki nazis um why did you delete the l...
1200,0,funny i dont feel blocked
140664,0,youre forgetting the towers were compromised b...
149618,0,some pictures i have uploaded some historical ...
104326,0,i am posting sourced content i am posting sour...
...,...,...
24778,1,yous a muthafin lie lifeasking pearls coreyema...
24779,0,0
24780,1,young buck wan na eat dat nigguh like i aint f...
24781,1,youu got wild bitches tellin you lies


In [19]:
df_balanced['toxic'].value_counts()

0    44163
1    32925
Name: toxic, dtype: int64

In [20]:
#!pip install transformers datasets evaluate accelerate peft

In [21]:
df_balanced

,toxic,comment_text
21447,0,note to wiki nazis um why did you delete the l...
1200,0,funny i dont feel blocked
140664,0,youre forgetting the towers were compromised b...
149618,0,some pictures i have uploaded some historical ...
104326,0,i am posting sourced content i am posting sour...
...,...,...
24778,1,yous a muthafin lie lifeasking pearls coreyema...
24779,0,0
24780,1,young buck wan na eat dat nigguh like i aint f...
24781,1,youu got wild bitches tellin you lies


In [22]:
import torch
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [23]:
toxicity_list=[]
for sen in df_balanced['toxic'].tolist():
  if sen==0:
    toxicity_list.append("toxic")
  else:
    toxicity_list.append("not-toxic")

In [24]:
# Sample data
data = {
    'text': df_balanced['comment_text'].tolist(),
    'label': toxicity_list
}


In [25]:
# Create DataFrame
df = pd.DataFrame(data)

df['text']=df['text'].astype(str)

In [26]:

unique_labels = df['label'].unique()

# Create a dictionary to map each label to a unique ID
label2id = {label: idx for idx, label in enumerate(unique_labels)}
df['label']=[label2id[label] for label in df.label]
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [27]:
import os
import warnings
import re
import argparse
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AdamW
from datasets import load_dataset, Dataset, DatasetDict
warnings.filterwarnings('ignore')

#accuracy=evaluate.load('accuracy')
def prepare_dataset(df):
  #df['text']=df['text']
  df['label']=df['label'].replace(label2id.keys(),label2id.values())

  dataset=DatasetDict()
  dataset['train']=Dataset.from_pandas(train_df[['text','label']].reset_index(drop=True))

  dataset['val']=Dataset.from_pandas(val_df[['text','label']].reset_index(drop=True))
  dataset['test']=Dataset.from_pandas(test_df[['text','label']].reset_index(drop=True))
  return dataset


# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


In [28]:
#model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id),id2label={y:x for x,y in label2id.items()},label2id=label2id)
model_name = "FacebookAI/roberta-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset=prepare_dataset(df)
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/53961 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/11563 [00:00<?, ? examples/s]

Map:   0%|          | 0/11564 [00:00<?, ? examples/s]

In [29]:
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model

In [30]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [31]:
# use the same Training args for all models
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    learning_rate=5e-5,
    num_train_epochs=2,
    per_device_train_batch_size=16,
)

In [32]:
def get_trainer(model):
      return  Trainer(
          model=model,
          args=training_args,
          train_dataset=tokenized_datasets["train"],
          eval_dataset=tokenized_datasets["test"],
          data_collator=data_collator,
      )


In [33]:
{y:x for x,y in label2id.items()}

{0: 'toxic', 1: 'not-toxic'}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id), id2label={y:x for x,y in label2id.items()}, label2id=label2id)

peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
peft_model = get_peft_model(model, peft_config)

print('PEFT Model')
peft_model.print_trainable_parameters()

peft_lora_finetuning_trainer = get_trainer(peft_model)

peft_lora_finetuning_trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PEFT Model
trainable params: 1,838,082 || all params: 357,199,876 || trainable%: 0.5145808057335384


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


In [ ]:
peft_model.save_pretrained("xlm-roberta-peft")